In [ ]:
import numpy as np  
import pandas as pd  
from datetime import timedelta
import datetime as dt
import lightgbm as lgb
import catboost as cbt
import datetime as dt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans 

In [ ]:
t0 = dt.datetime.now()
test = pd.read_csv('./data/test.csv')
train = pd.read_csv('./data/train.csv')
test_1 = test.copy()

In [ ]:
# 数据预处理,先删除一些没有奇异值
train = train[train['passenger_count'] > 0]   

In [ ]:
# 计算时间特征,通过上述的分析我们发现时间特征是非常重要的一个维度，所以我们选择将其从数据中抽取出来
train['pickup_datetime'] = pd.to_datetime(train.pickup_datetime)
test['pickup_datetime'] = pd.to_datetime(test.pickup_datetime)
train['pickup_date'] = train['pickup_datetime'].dt.date
test['pickup_date'] = test['pickup_datetime'].dt.date
train['dropoff_datetime'] = pd.to_datetime(train.dropoff_datetime)

train['pickup_weekday'] = train['pickup_datetime'].dt.weekday
train['pickup_hour_weekofyear'] = train['pickup_datetime'].dt.weekofyear
train['pickup_hour'] = train['pickup_datetime'].dt.hour
train['pickup_minute'] = train['pickup_datetime'].dt.minute
train['pickup_dt'] = (train['pickup_datetime'] - train['pickup_datetime'].min()).dt.total_seconds()
train['pickup_week_hour'] = train['pickup_weekday'] * 24 + train['pickup_hour']

test['pickup_weekday'] = test['pickup_datetime'].dt.weekday
test['pickup_hour_weekofyear'] = test['pickup_datetime'].dt.weekofyear
test['pickup_hour'] = test['pickup_datetime'].dt.hour
test['pickup_minute'] = test['pickup_datetime'].dt.minute
test['pickup_dt'] = (test['pickup_datetime'] - train['pickup_datetime'].min()).dt.total_seconds()
test['pickup_week_hour'] = test['pickup_weekday'] * 24 + test['pickup_hour']

train['pickup_dayofyear'] = train['pickup_datetime'].dt.dayofyear
test['pickup_dayofyear'] = test['pickup_datetime'].dt.dayofyear

In [ ]:
# 计算出发点到目的地的角度方向,参考的是wiki的内容
def bearing_array(lat1, lng1, lat2, lng2):
    AVG_EARTH_RADIUS = 6378.137  # in km
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

train['direction'] = bearing_array(train['pickup_latitude'].values, train['pickup_longitude'].values, 
                                          train['dropoff_latitude'].values, train['dropoff_longitude'].values)

test['direction'] = bearing_array(test['pickup_latitude'].values, test['pickup_longitude'].values, 
                                         test['dropoff_latitude'].values, test['dropoff_longitude'].values)

In [ ]:

# 下面的这些特征的提取来源于kaggle论坛的讨论,因为大家认为地图上的欧几里得距离并不能真实反映实际的情况，所以应该
# 加入街道距离等.
def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

def Dummy_MahaDis(lat1, lng1, lat2, lng2):
    tmp1 = haversine_array(lat1, lng1, lat1, lng2)
    tmp2 = haversine_array(lat1, lng1, lat2, lng1)
    return tmp1 + tmp2

train['distance_haversine'] = haversine_array(train['pickup_latitude'].values, train['pickup_longitude'].values, train['dropoff_latitude'].values, train['dropoff_longitude'].values)
train['distance_dummy_manhattan'] = Dummy_MahaDis(train['pickup_latitude'].values, train['pickup_longitude'].values, train['dropoff_latitude'].values, train['dropoff_longitude'].values)


test['distance_haversine'] = haversine_array(test['pickup_latitude'].values, test['pickup_longitude'].values, test['dropoff_latitude'].values, test['dropoff_longitude'].values)
test['distance_dummy_manhattan'] = Dummy_MahaDis(test['pickup_latitude'].values, test['pickup_longitude'].values, test['dropoff_latitude'].values, test['dropoff_longitude'].values)

# 计算中间的经纬度，参考notebook:https://www.kaggle.com/mubashir44/xgboost-with-kfold-valid-lb-0-368
train['center_latitude'] = (train['pickup_latitude'].values + train['dropoff_latitude'].values) / 2
train['center_longitude'] = (train['pickup_longitude'].values + train['dropoff_longitude'].values) / 2
test['center_latitude'] = (test['pickup_latitude'].values + test['dropoff_latitude'].values) / 2
test['center_longitude'] = (test['pickup_longitude'].values + test['dropoff_longitude'].values) / 2


In [ ]:
# 计算pca的距离，参考notebook:https://www.kaggle.com/gaborfodor/from-eda-to-the-top-lb-0-367,因为真实的维度就只有4维,
# 所以我们这边选用其中的三维.
coords = np.vstack((train[['pickup_latitude', 'pickup_longitude']].values,
                    train[['dropoff_latitude', 'dropoff_longitude']].values,
                    test[['pickup_latitude', 'pickup_longitude']].values,
                    test[['dropoff_latitude', 'dropoff_longitude']].values))

pca = PCA().fit(coords)

train['pickup_pca0'] = pca.transform(train[['pickup_latitude', 'pickup_longitude']])[:, 0]
train['pickup_pca1'] = pca.transform(train[['pickup_latitude', 'pickup_longitude']])[:, 1] 
train['dropoff_pca0'] = pca.transform(train[['dropoff_latitude', 'dropoff_longitude']])[:, 0]
train['dropoff_pca1'] = pca.transform(train[['dropoff_latitude', 'dropoff_longitude']])[:, 1] 
test['pickup_pca0'] = pca.transform(test[['pickup_latitude', 'pickup_longitude']])[:, 0]
test['pickup_pca1'] = pca.transform(test[['pickup_latitude', 'pickup_longitude']])[:, 1] 
test['dropoff_pca0'] = pca.transform(test[['dropoff_latitude', 'dropoff_longitude']])[:, 0]
test['dropoff_pca1'] = pca.transform(test[['dropoff_latitude', 'dropoff_longitude']])[:, 1] 


In [ ]:
# 计算pca的曼哈顿距离,参考的notebook：notebook:https://www.kaggle.com/gaborfodor/from-eda-to-the-top-lb-0-367
train.loc[:, 'pca_manhattan'] = np.abs(train['dropoff_pca1'] - train['pickup_pca1']) + np.abs(train['dropoff_pca0'] - train['pickup_pca0'])
test.loc[:, 'pca_manhattan'] = np.abs(test['dropoff_pca1'] - test['pickup_pca1']) + np.abs(test['dropoff_pca0'] - test['pickup_pca0'])

In [ ]:
## 对store_and_fwd_flag进行编码，主要是为了方便我们的model的运行
train['store_and_fwd_flag'] = train['store_and_fwd_flag'].map(lambda x: 0 if x == 'N' else 1)
test['store_and_fwd_flag'] = test['store_and_fwd_flag'].map(lambda x: 0 if x == 'N' else 1)

In [ ]:
# 每次采样300000个样本,然后进行K-means的聚类操作,这边的30000可以自己进行调试.
indexes = np.random.permutation(len(coords))[:300000]
kmeans = MiniBatchKMeans(n_clusters=50, batch_size=10000).fit(coords[indexes])
train.loc[:, 'pickup_cluster'] = kmeans.predict(train[['pickup_latitude', 'pickup_longitude']])
train.loc[:, 'dropoff_cluster'] = kmeans.predict(train[['dropoff_latitude', 'dropoff_longitude']])
test.loc[:, 'pickup_cluster'] = kmeans.predict(test[['pickup_latitude', 'pickup_longitude']])
test.loc[:, 'dropoff_cluster'] = kmeans.predict(test[['dropoff_latitude', 'dropoff_longitude']])
t1 = dt.datetime.now() 

In [ ]:
# 这两个数据是后来其他人分享的,所以我们直接将其加入到我们的模型中来
train_part_1 = pd.read_csv('./data/fastest_routes_train_part_1.csv', usecols=['id', 'total_distance', 'total_travel_time',  'number_of_steps', ])
train_part_2 = pd.read_csv('./data/fastest_routes_train_part_2.csv', usecols=['id', 'total_distance', 'total_travel_time', 'number_of_steps'])
test_street_info = pd.read_csv('./data/fastest_routes_test.csv',usecols=['id', 'total_distance', 'total_travel_time', 'number_of_steps'])
train_street_info = pd.concat((train_part_1, train_part_2))
train = train.merge(train_street_info, how='left', on='id')
test = test.merge(test_street_info, how='left', on='id')

In [ ]:
# 为了方便我们的指标的计算,所以此处我们将其换算成log的形式
train['log_trip_duration'] = np.log(train['trip_duration'].values + 1)

In [ ]:
# 准备训练和测试数据
feature_names = list(train.columns) 
not_use_names = ['id', 'log_trip_duration', 'trip_duration', 'dropoff_datetime', 'pickup_date', 'pickup_datetime', 'date']
feature_names = [x for x in train.columns if f not in not_use_names]

In [ ]:
# xgboost训练得到模型1
train_X = train[feature_names].values
train_y = np.log(train['trip_duration'].values + 1)  
 
test_X = test[feature_names].values
dtrain = xgb.DMatrix(train_X, label=train_y) 
dtest = xgb.DMatrix(test_X) 
xgb_pars = { 'booster' : 'gbtree','min_child_weight': 12, 'eta': 0.05, 'max_depth': 10, 'colsample_bytree': 0.9,
            'subsample': 0.9, 'lambda': 5, 'nthread': -1, 'silent': 1, 'eval_metric': 'rmse', 'objective': 'reg:linear'}    

model = xgb.train(xgb_pars, dtrain, 500, maximize=False, verbose_eval=15)
res_xgb = model.predict(dtest)

In [ ]:
# catboost训练得到模型2
cat = cbt.CatBoostRegressor(iterations= 8000, learning_rate= 0.1, depth = 6, random_seed=1)
cat.fit(train_X,train_y) 
res_cat = cat.predict(test_X)  

In [ ]:
# lightGBM训练得到模型3
lgb_train = lgb.Dataset(train_X, train_y)
 
params = {'metric': 'rmse', 'learning_rate' : 0.05, 'num_leaves': 311, 
         'feature_fraction': 0.9,'bagging_fraction':0.9,'bagging_freq':5,'min_data_in_leaf': 500}
lgb_model = lgb.train(params, lgb_train, num_boost_round = 350)
res_lgb = lgb_model.predict(lgb.Dataset(test_X.values))

In [ ]:
# 模型加权集成
final_res = 0.45 * res_xgb + 0.3 * res_cat + 0.25 * res_lgb

test['trip_duration'] = np.exp(final_res) - 1
test.loc[test['trip_duration']<0,'trip_duration'] = 0
test[['id', 'trip_duration']].to_csv('./predict/submit.csv', index=False